## Feature extraction

In [57]:
# Imports
from gensim.models import FastText

%run preprocessing.ipynb

15
Char [['m', 's', '>', 'l', 'p'], ['w', 'm', 'n'], ['H', 'n', 'v'], ['w', 'h', 'w'], ['q', 'A', 'd', 'r'], ['E', 'l', 'Y'], ['A', 'l', '<', 'T', 'E', 'A', 'm'], ['>', 'w'], ['A', 'l', 'k', 's', 'w', 'p'], ['>', 'w'], ['A', 'l', 'E', 't', 'q'], ['v', 'm'], ['A', 'f', 't', 'q', 'r'], ['f', 'E', 'j', 'z'], ['E', 'n'], ['k', 'l'], ['*', 'l', 'k'], ['l', 'm'], ['y', 'j', 'z', 'h'], ['A', 'l', 'S', 'w', 'm'], ['>', 'S', 'l', 'A']]
Diac [['a', 'o', 'a', 'a', 'N'], ['a', 'a', 'o'], ['a', 'i', 'a'], ['a', 'u', 'a'], ['a', ' ', 'i', 'N'], ['a', 'a', ' '], [' ', ' ', 'i', 'o', 'a', ' ', 'i'], ['a', 'o'], [' ', 'o', 'i', 'o', 'a', 'i'], ['a', 'o'], [' ', 'o', 'i', 'o', 'i'], ['u', '~a'], [' ', 'o', 'a', 'a', 'a'], ['a', 'a', 'a', 'a'], ['a', 'o'], ['u', '~i'], ['a', 'i', 'a'], ['a', 'o'], ['u', 'o', 'i', 'i'], [' ', ' ', '~a', 'o', 'u'], ['a', 'o', 'F', ' ']]


### Read the corpus

In [58]:
def readFile(path):
	sentences = []
	with open(path, 'r', encoding='utf-8') as file:
		for line in file:
			sentences.append(line.strip())

	return sentences

PATH = "../dataset/train.txt"
corpus = readFile(PATH)

### FastText

In [59]:
data = []
for sentence in corpus:
    sentence = run_buckwalter(sentence.strip())
    char_list, _ = extract_labels(sentence)

    char_list = ["".join(sen) for sen in char_list]
    data.append(char_list)

# Defining values for parameters
embedding_size = 100
window_size = 20
min_word = 5
down_sampling = 1e-2

fast_Text_model = FastText(
                            vector_size=embedding_size,
                            window=window_size,
                            min_count=min_word,
                            sample=down_sampling,
                            workers=4,
                            epochs=50,
                            seed=42,
                            sg=1)
fast_Text_model.build_vocab(data, progress_per=10000)
fast_Text_model.train(data, total_examples=fast_Text_model.corpus_count, epochs=50, report_delay=1)

(98820800, 105103400)

In [74]:
fast_Text_model.save("./models/ft_model")
print(fast_Text_model.wv[buckwalter.transliterate("ياكل")])
print(fast_Text_model.wv.similarity(buckwalter.transliterate("احمد"), buckwalter.transliterate("محمد")))

[-0.17587607  0.02877168  0.04709973  0.34052846 -0.10787471  0.03396543
 -0.11186929  0.03204577  0.02601715 -0.04619154 -0.64464664  0.23255761
 -0.20373139 -0.03959081  0.08147166  0.05366697  0.05857076 -0.03167843
 -0.06125082  0.17182784 -0.14936416  0.07375959 -0.27469558 -0.34764272
  0.29371625 -0.10324     0.08420175 -0.05359089  0.08896402 -0.04694974
  0.3940764   0.08949303 -0.5643163  -0.2209296   0.33949855  0.14446706
  0.13378167 -0.15082327  0.15100697 -0.05039954  0.28573117  0.02301891
  0.01095174 -0.22188954 -0.2218215  -0.28840852 -0.01482417 -0.0590201
  0.36846104 -0.03828136 -0.39380917  0.06536474  0.321336    0.25974402
  0.27468032 -0.1842511  -0.28447118  0.0251733  -0.09457348  0.19323996
  0.25880393 -0.04231093 -0.12811872  0.352846    0.5898262  -0.15524375
  0.19650614 -0.0900159  -0.19504753 -0.3754987   0.2750499  -0.06968006
  0.26242155  0.08634424  0.2556117  -0.35728014 -0.35281286  0.2248184
  0.05599839  0.34907785  0.10232218  0.3731012  -0.0

### Main

In [78]:
# word_embeddings_fasttext()
loaded_model = FastText.load("./models/ft_model")
print(loaded_model.wv.similarity(buckwalter.transliterate("احمد"), buckwalter.transliterate("محمد")))

0.56691873
